In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('../mtr-datasets/atp7d.arff')
data = pd.DataFrame(data)
data.head()

"'departwb'"  "'departdaytype'"  "'fetchdays'"  "'fetchdow'"  "'dtd'"  \
0          72.0                1.0           60.0           3.0     12.0   
1          79.0                1.0           69.0           5.0     10.0   
2          79.0                1.0           70.0           6.0      9.0   
3          79.0                1.0           71.0           7.0      8.0   
4          79.0                1.0           72.0           1.0      7.0   

   "'ALLminpA'"  "'ALLminpA'l1"  "'ALLminpA'l2"  "'ALLminpA'l7"  "'ALLavgpA'"  \
0         359.0           372.0           351.0           317.0        615.96   
1         336.0           298.0           298.0           320.0        616.73   
2         336.0           336.0           298.0           320.0        628.87   
3         336.0           336.0           336.0           330.0        632.73   
4         336.0           336.0           336.0           330.0        635.47   

   ...  "fdisoweekday4"  "fdisoweekday5"  "fdisoweekday6"  "fdisoweekday7"  \
0  ...              0.0              0.0              0.0              0.0   
1  ...              0.0              1.0              0.0              0.0   
2  ...              0.0              0.0              1.0              0.0   
3  ...              0.0              0.0              0.0              1.0   
4  ...              0.0              0.0              0.0              0.0   

   LBL+ALLminpA+bt7d_000  LBL+ALLminp0+bt7d_000  LBL+aDLminpA+bt7d_000  \
0                  326.0                  675.0                  626.0   
1                  315.0                  675.0                  631.0   
2                  315.0                  675.0                  631.0   
3                  315.0                  675.0                  631.0   
4                  315.0                 1182.0                  720.0   

   LBL+aCOminpA+bt7d_000  LBL+aFLminpA+bt7d_000  LBL+aUAminpA+bt7d_000  
0                  545.0                  326.0                  545.0  
1                  587.0                  315.0                  587.0  
2                  587.0                  315.0                  587.0  
3                  587.0                  315.0                  587.0  
4                  720.0                  315.0                  720.0  

[5 rows x 417 columns]

In [4]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=100)

In [5]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (207, 411)
Train_Y:  (207, 6)
Test_X:  (89, 411)
Test_Y:  (89, 6)


In [6]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))


## Chain Model type 1


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [8]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='glorot_uniform'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

In [9]:
from time import time

In [10]:
#Model chain
chain = []

#Training
X_feed = X_train.copy()
t1 = time()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=500,batch_size=50)
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)

t2 = time()

print("Time taken: ",(t2-t1))

Training chain node  0
Shapes:
 X = (207, 411) 
 Y = (207, 1)
Epoch 1/500
5/5 [==============================] - 0s 2ms/step - loss: 0.7529 - mse: 0.7529
Epoch 2/500
5/5 [==============================] - 0s 1ms/step - loss: 0.8246 - mse: 0.8246
Epoch 3/500
5/5 [==============================] - 0s 1ms/step - loss: 0.4807 - mse: 0.4807
Epoch 4/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2730 - mse: 0.2730
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.2038 - mse: 0.2038
Epoch 6/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1838 - mse: 0.1838
Epoch 7/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1193 - mse: 0.1193
Epoch 8/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0896 - mse: 0.0896
Epoch 9/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0651 - mse: 0.0651
Epoch 10/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0596 - mse: 0.0596
Epoch 11/500
5/5 

5/5 [==============================] - 0s 1ms/step - loss: 6.8309e-04 - mse: 6.8309e-04
Epoch 86/500
5/5 [==============================] - 0s 1ms/step - loss: 3.7336e-04 - mse: 3.7336e-04
Epoch 87/500
5/5 [==============================] - 0s 1ms/step - loss: 4.0035e-04 - mse: 4.0035e-04
Epoch 88/500
5/5 [==============================] - 0s 1ms/step - loss: 4.4390e-04 - mse: 4.4390e-04
Epoch 89/500
5/5 [==============================] - 0s 1ms/step - loss: 3.3600e-04 - mse: 3.3600e-04
Epoch 90/500
5/5 [==============================] - 0s 1ms/step - loss: 3.6331e-04 - mse: 3.6331e-04
Epoch 91/500
5/5 [==============================] - 0s 1ms/step - loss: 2.3506e-04 - mse: 2.3506e-04
Epoch 92/500
5/5 [==============================] - 0s 1ms/step - loss: 2.5935e-04 - mse: 2.5935e-04
Epoch 93/500
5/5 [==============================] - 0s 1ms/step - loss: 2.5187e-04 - mse: 2.5187e-04
Epoch 94/500
5/5 [==============================] - 0s 1ms/step - loss: 3.2372e-04 - mse: 3.2372e-04
Epo

5/5 [==============================] - 0s 1ms/step - loss: 0.0142 - mse: 0.0142
Epoch 171/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096
Epoch 172/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102
Epoch 173/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0148 - mse: 0.0148
Epoch 174/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105
Epoch 175/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0053 - mse: 0.0053
Epoch 176/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0043 - mse: 0.0043
Epoch 177/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037
Epoch 178/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0039 - mse: 0.0039
Epoch 179/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 180/500
5/5 [==============================] - 0s 1ms/step - loss: 0

5/5 [==============================] - 0s 1ms/step - loss: 8.6875e-04 - mse: 8.6875e-04
Epoch 254/500
5/5 [==============================] - 0s 1ms/step - loss: 4.3861e-04 - mse: 4.3861e-04
Epoch 255/500
5/5 [==============================] - 0s 1ms/step - loss: 6.7958e-04 - mse: 6.7958e-04
Epoch 256/500
5/5 [==============================] - 0s 1ms/step - loss: 5.7708e-04 - mse: 5.7708e-04
Epoch 257/500
5/5 [==============================] - 0s 2ms/step - loss: 5.7962e-04 - mse: 5.7962e-04
Epoch 258/500
5/5 [==============================] - 0s 1ms/step - loss: 5.3235e-04 - mse: 5.3235e-04
Epoch 259/500
5/5 [==============================] - 0s 1ms/step - loss: 9.0707e-04 - mse: 9.0707e-04
Epoch 260/500
5/5 [==============================] - 0s 2ms/step - loss: 6.4292e-04 - mse: 6.4292e-04
Epoch 261/500
5/5 [==============================] - 0s 1ms/step - loss: 5.6739e-04 - mse: 5.6739e-04
Epoch 262/500
5/5 [==============================] - 0s 1ms/step - loss: 6.1809e-04 - mse: 6.180

5/5 [==============================] - 0s 1ms/step - loss: 0.0704 - mse: 0.0704
Epoch 336/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0494 - mse: 0.0494
Epoch 337/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0390 - mse: 0.0390
Epoch 338/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0291 - mse: 0.0291
Epoch 339/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0536 - mse: 0.0536
Epoch 340/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0854 - mse: 0.0854
Epoch 341/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0655 - mse: 0.0655
Epoch 342/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0348 - mse: 0.0348
Epoch 343/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0370 - mse: 0.0370
Epoch 344/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0305 - mse: 0.0305
Epoch 345/500
5/5 [==============================] - 0s 2ms/step - loss: 0

...

5/5 [==============================] - 0s 1ms/step - loss: 0.0131 - mse: 0.0131
Epoch 94/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0085 - mse: 0.0085
Epoch 95/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081
Epoch 96/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108
Epoch 97/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081
Epoch 98/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0055 - mse: 0.0055
Epoch 99/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0038 - mse: 0.0038
Epoch 100/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0037 - mse: 0.0037
Epoch 101/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0040 - mse: 0.0040
Epoch 102/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0031 - mse: 0.0031
Epoch 103/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0021 

5/5 [==============================] - 0s 1ms/step - loss: 5.3799e-04 - mse: 5.3799e-04
Epoch 176/500
5/5 [==============================] - 0s 1ms/step - loss: 7.5505e-04 - mse: 7.5505e-04
Epoch 177/500
5/5 [==============================] - 0s 1ms/step - loss: 5.1298e-04 - mse: 5.1298e-04
Epoch 178/500
5/5 [==============================] - 0s 2ms/step - loss: 5.1941e-04 - mse: 5.1941e-04
Epoch 179/500
5/5 [==============================] - 0s 2ms/step - loss: 9.3249e-04 - mse: 9.3249e-04
Epoch 180/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0010 - mse: 0.0010
Epoch 181/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 182/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 183/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0019 - mse: 0.0019
Epoch 184/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 185/500
5/5 [===============

5/5 [==============================] - 0s 1ms/step - loss: 0.0026 - mse: 0.0026    
Epoch 262/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027
Epoch 263/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0027 - mse: 0.0027
Epoch 264/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025
Epoch 265/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0024 - mse: 0.0024
Epoch 266/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0020 - mse: 0.0020
Epoch 267/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 268/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0039 - mse: 0.0039
Epoch 269/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0026 - mse: 0.0026
Epoch 270/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0027 - mse: 0.0027
Epoch 271/500
5/5 [==============================] - 0s 2ms/step - los

5/5 [==============================] - 0s 2ms/step - loss: 1.4249e-04 - mse: 1.4249e-04
Epoch 346/500
5/5 [==============================] - 0s 1ms/step - loss: 1.2292e-04 - mse: 1.2292e-04
Epoch 347/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0756e-04 - mse: 3.0756e-04
Epoch 348/500
5/5 [==============================] - 0s 1ms/step - loss: 2.5584e-04 - mse: 2.5584e-04
Epoch 349/500
5/5 [==============================] - 0s 1ms/step - loss: 1.9970e-04 - mse: 1.9970e-04
Epoch 350/500
5/5 [==============================] - 0s 1ms/step - loss: 2.8827e-04 - mse: 2.8827e-04
Epoch 351/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023
Epoch 352/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 353/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 354/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0045 - mse: 0.0045
Epoch 355/500
5/5 

5/5 [==============================] - 0s 1ms/step - loss: 0.0065 - mse: 0.0065
Epoch 431/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0045 - mse: 0.0045
Epoch 432/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0048 - mse: 0.0048
Epoch 433/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0039 - mse: 0.0039
Epoch 434/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050
Epoch 435/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0064 - mse: 0.0064
Epoch 436/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0066 - mse: 0.0066
Epoch 437/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0029 - mse: 0.0029
Epoch 438/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022
Epoch 439/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0044 - mse: 0.0044
Epoch 440/500
5/5 [==============================] - 0s 1ms/step - loss: 0

In [14]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


In [15]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [16]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))

RMSE:  44.08863027328879
MAE:  26.23347567440419
R_Squared:  0.7284513003012482
